In [ ]:
import io, os
input=io.BytesIO(os.read(0,os.fstat(0).st_size)).readline

class DSU:
  def __init__(self, n):
    self.P = list(range(n))
    self.s = [1] * n

  def find(self, a):
    _a = a
    while a != self.P[a]:
      a = self.P[a]
    while _a != a:
      self.P[_a], _a = a, self.P[_a]
    return a

  def union(self, a, b):
    a, b = self.find(a), self.find(b)
    if a != b:
      if self.s[a] < self.s[b]:
        a, b = b, a

      self.P[b] = a
      self.s[a] += self.s[b]

def sol() :
  N, M, K = map(int, input().split())
  dsu = DSU(N)
  L = [*map(int, input().split())]
  L.sort()
  iL = {v: i for i, v in enumerate(L)}
  L2 = [*map(int, input().split())]
  
  ans = []
  for q in L2 :
    lo=-1
    hi=N-1
    while lo + 1 < hi:
      mid = (lo + hi) // 2
      if dsu.find(mid) == mid and L[dsu.find(mid)] > q : #이미 차지되지 않았으며, 자기 자신이 부모이면 범위 축소
        hi = mid
        res = mid
      else:
        lo = mid
    dsu.union(res, q)
    ans.append(L[res])
  print(*ans)

sol()

### 아이디어
- 덱초기화 DP로 접근했다가 답이 안보여서 태그를 봤다.
  - 이분탐색, 분리집합.
- M개의 카드에 대해 분리집합을 만들고, 쿼리가 들어왔을 때 그 숫자에 해당하는 숫자보다 큰 인덱스 중에 가장 작은 인덱스를 반환하도록 구현해본 코드였으나, 답이 나오지 않았다.
  - 이분탐색의 특성상 연속한 구간의 논리값이 같다가 나머지 구간의 논리값이 그것과 반대여야 하고, 이러한 반전이 한번밖에 없어야 하는데 이 구현은 그렇게 하지 못한다.
- q에 대해서 가장 가까운 인덱스를 찾는 것을 이분탐색으로 하고, find에 대한 값이 가능한 가장 낮은(내야하는) 카드번호로 하게 하는 방법이 떠올랐다.

In [ ]:
import io, os, sys, bisect
input=io.BytesIO(os.read(0,os.fstat(0).st_size)).readline

class DSU:
  def __init__(self, n):
    self.P = list(range(n))
    self.s = [1] * n

  def find(self, a):
    _a = a
    while a != self.P[a]:
      a = self.P[a]
    while _a != a:
      self.P[_a], _a = a, self.P[_a]
    return a

  def union(self, a, b):
    a, b = self.find(a), self.find(b)
    if a != b:
      if self.s[a] < self.s[b]:
        a, b = b, a

      self.P[b] = a
      self.s[a] += self.s[b]

def sol() :
  N, M, K = map(int, input().split())
  L = [*map(int, input().split())]
  dsu = DSU(M+1)
  L.sort()

  ans = []
  Q = [*map(int, input().split())]
  for q in Q :
    idx = bisect.bisect_right(L, q)
    x = dsu.find(idx) #비어 있는 부모의 인덱스
    tmp = x + 1
    while dsu.P[dsu.find(tmp)] != tmp : #차지되지 않은 부모를 찾을 때 까지
      tmp += 1
    ans.append(L[tmp-1])
    dsu.union(dsu.find(tmp), x)

  sys.stdout.write('\n'.join(map(str, ans)))

sol()

- WA. 그래도 대충 접근은 이쪽인 것 같은 희망을 얻었다.

### 풀이
- 아이디어는 맞았지만 정답을 낼 수 없었다. 이유는 Union by rank의 특성 때문.
- 문제에서 필요한 정점 번호는 항상 증가하는 방향으로 선택되어야 한다.
  - 그런데, Union by rank는 이러한 원소가 가리키는 방향의 단조 증가성을 없앤다. 따라서 Union by rank를 사용해선 올바른 정답을 구할 수 없다.
    - 자식의 숫자가 많은 쪽으로 부모를 붙이는 특성상, 정점 번호가 낮은 것이 정점 번호가 높은 것을 자식으로 두는 경우가 생긴다.

### 정답 코드
- 시간복잡도 $\Omicron(M \log M)$

In [ ]:
import io, os, sys, bisect
input=io.BytesIO(os.read(0,os.fstat(0).st_size)).readline

class DSU:
  def __init__(self, n):
    self.P = list(range(n))
    self.s = [1] * n

  def find(self, a):
    _a = a
    while a != self.P[a]:
      a = self.P[a]
    while _a != a:
      self.P[_a], _a = a, self.P[_a]
    return a

  def union(self, a, b):
    a, b = self.find(a), self.find(b)
    if a != b:
      # if self.s[a] < self.s[b]:
      #   a, b = b, a

      self.P[b] = a
      self.s[a] += self.s[b]

def sol() :
  N, M, K = map(int, input().split())
  L = [*map(int, input().split())]
  dsu = DSU(M+1)
  L.sort()

  ans = []
  Q = [*map(int, input().split())]
  for q in Q :
    idx = bisect.bisect_right(L, q)
    x = dsu.find(idx) #비어 있는 부모의 인덱스
    ans.append(L[x])
    dsu.union(x+1, x)

  sys.stdout.write('\n'.join(map(str, ans)))

sol()